In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, measure, morphology
import math
from skimage.measure import label, regionprops
from PIL import Image
import colorsys
from skimage import img_as_ubyte, exposure
from os import listdir
from os.path import isfile, join
import os
from qualityCheck import nucleiCheckTA, nucleiCheckTA1, nucleiCheckTANOQC

In [11]:
cytoSegPath = "/Users/jones/Downloads/pruningSCImages/preprocessedImages/cellShape/predict/230724_EdU_pruning_3-12_cellShape.tif"
nucSegPath = "/Users/jones/Downloads/pruningSCImages/segmentation/230724_EdU_pruning_3-12_nuclei_seg.npy"
nucPath = "/Users/jones/Downloads/pruningSCImages/preprocessedImages/nuclei/230724_EdU_pruning_3-12_nuclei.tif"
cytoPath = "/Users/jones/Downloads/pruningSCImages/preprocessedImages/cellShape/230724_EdU_pruning_3-12_cellShape.tif"
savePath = '/Users/jones/Downloads/pruningSCImages/cellShapeSeg'
saveName = '230724_EdU_pruning_3-12_cellShape_OLDseg.npy'

In [12]:
cytoMasks = nucleiCheckTA(cytoSegPath, nucSegPath, 3, 0.8)
originalImage = cv2.imread(cytoPath)
nucleiImage = cv2.imread(nucPath)
originalImage2 = originalImage + nucleiImage

cellposeFile = {}
zeros = np.zeros(originalImage.shape, dtype="uint8")

dapi8bitNormalized= img_as_ubyte(exposure.rescale_intensity(originalImage2))
cellSize8bitNormalized = img_as_ubyte(exposure.rescale_intensity(originalImage))
dapi8bitNormalized = np.asarray(dapi8bitNormalized)
dapi8bitNormalized = dapi8bitNormalized
dapi8bitNormalized = dapi8bitNormalized.astype(cellSize8bitNormalized.dtype)
merged8bitNormalized = cv2.merge([dapi8bitNormalized[:,:,0], cellSize8bitNormalized[:,:,0], cellSize8bitNormalized[:,:,0]])

outlines = cv2.imread(cytoSegPath)[:,:,0]
#save in dic
cellposeFile['masks'] = cytoMasks
cellposeFile['img'] = merged8bitNormalized
cellposeFile['outlines'] = outlines


os.chdir(savePath)
np.save(saveName, cellposeFile)

In [ ]:
for cytoFolder in cytoFolders:
    os.chdir(cytoFolder)
    files = [f for f in listdir(cytoFolder) if isfile(join(cytoFolder, f)) if ".DS_Store" not in f if ".tif" in f if "._" not in f]
    aorta = cytoFolder.split("/")[-2]
    for file in files:
        aorta = cytoFolder.split("/")[-2]
        age = cytoFolder.split("/")[-3]
        #nucFile = file.split("_cellSize")[0] + "_" +  file.split("_cellSize")[1].split(".tif")[0] + "_cellSize_seg.npy"
        nucFile = file.split("_cellSize")[0] + "_nuclei" + file.split(".tif")[0].split("cellSize")[-1] + "_seg.npy"
        nucleiPath = join(nucBasePath, age, aorta, nucFile)
        cytoMasks = nucleiCheckTA(join(cytoFolder, file), nucleiPath, 3, 0.8)

        originalImage = cv2.imread(join(originalImageBase, age, aorta, file))
        #nucIm = file.split("_cellSize")[0] + "_" +  file.split("_cellSize")[1].split(".tif")[0] + "_cellSize.tif"
        nucIm = file.split("_cellSize")[0] + "_nuclei" + file.split(".tif")[0].split("cellSize")[-1] + ".tif"
        nucleiImage = cv2.imread(join(nucleiImageBase, age, aorta, nucIm))
        originalImage2 = originalImage + nucleiImage

        cellposeFile = {}
        zeros = np.zeros(originalImage.shape, dtype="uint8")

        dapi8bitNormalized= img_as_ubyte(exposure.rescale_intensity(originalImage2))
        cellSize8bitNormalized = img_as_ubyte(exposure.rescale_intensity(originalImage))
        dapi8bitNormalized = np.asarray(dapi8bitNormalized)
        dapi8bitNormalized = dapi8bitNormalized
        dapi8bitNormalized = dapi8bitNormalized.astype(cellSize8bitNormalized.dtype)
        merged8bitNormalized = cv2.merge([dapi8bitNormalized[:,:,0], cellSize8bitNormalized[:,:,0], cellSize8bitNormalized[:,:,0]])

        outlines = cv2.imread(join(cytoFolder, file))[:,:,0]
        #save in dic
        cellposeFile['masks'] = cytoMasks
        cellposeFile['img'] = merged8bitNormalized
        cellposeFile['outlines'] = outlines


        os.chdir(cytoFolder)
        os.chdir(os.path.normpath(os.getcwd() + os.sep + os.pardir))
        np.save(file.split(".tif")[0] + "_NEWseg.npy", cellposeFile)